In [1]:
import os

In [2]:
import logging

logging.basicConfig(
    # filename=__file__.replace('.py', '.log'),
    level=logging.getLevelName("INFO"),
    format="%(asctime)s [%(levelname)s] [%(module)s] %(message)s",
)

log = logging.getLogger(__name__)

In [3]:
import sys


def in_kaggle():
    return "kaggle_web_client" in sys.modules


log.info(f"in_kaggle: {in_kaggle()}")

2023-02-21 12:06:48,233 [INFO] [365646948] in_kaggle: False


In [4]:
##### INSERT SOURCE CODE HERE FOR SUBMISSION #####

# PLACE HOLDER

In [5]:
if in_kaggle():
    !cp -r /kaggle/input/graphnet-and-dependencies/software .
    !pip install /kaggle/working/software/dependencies/torch-1.11.0+cu115-cp37-cp37m-linux_x86_64.whl
    !pip install /kaggle/working/software/dependencies/torch_cluster-1.6.0-cp37-cp37m-linux_x86_64.whl
    !pip install /kaggle/working/software/dependencies/torch_scatter-2.0.9-cp37-cp37m-linux_x86_64.whl
    !pip install /kaggle/working/software/dependencies/torch_sparse-0.6.13-cp37-cp37m-linux_x86_64.whl
    !pip install /kaggle/working/software/dependencies/torch_geometric-2.0.4.tar.gz
    !cd software/graphnet;pip install --no-index --find-links="/kaggle/working/software/dependencies" -e .[torch]

    # https://www.kaggle.com/speeddemon/install-hydra-offline-from-dataset
    !cp -r /kaggle/input/hydracore105 /kaggle/working
    !mv /kaggle/working/hydracore105/antlr4-python3-runtime-4.8.tar.gz.tmp /kaggle/working/hydracore105/antlr4-python3-runtime-4.8.tar.gz
    !ls /kaggle/working/hydracore105
    !pip install -qq /kaggle/working/hydracore105/* --ignore-installed PyYAML

    sys.path.append("../input/omegaconf/omegaconf-master")
    omega_conf_dir = "config"

else:
    %cd ..

    omega_conf_dir = "./config"

/home/jovyan/ice-cube


In [6]:
from omegaconf import OmegaConf

c_main = OmegaConf.load(os.path.join(omega_conf_dir, "main.yaml"))
c_data = OmegaConf.load(os.path.join(omega_conf_dir, "data.yaml"))
c = OmegaConf.merge(c_main, c_data)

c.settings.debug = False
c.wandb.enabled = False
c.settings.is_training = False

In [7]:
if in_kaggle():
    c.data.dir.pretrained = "/kaggle/input/dynedge-pretrained"

In [8]:
if c.settings.is_training:
    database_path = os.path.join(c.data.dir.working, "train_db.db")
else:
    database_path = os.path.join(c.data.dir.working, "test_db.db")

log.info(database_path)

2023-02-21 12:06:48,404 [INFO] [2742257138] ./test_db.db


In [9]:
!rm -f {database_path}
!python 00-create-db.py

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
graphnet: INFO     2023-02-21 12:06:50 - get_logger - Writing log to logs/graphnet_20230221-120650.log
graphnet: WARNING  2023-02-21 12:06:50 - warn_once - `icecube` not available. Some functionality may be missing.
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")
2023-02-21 12:06:52,283 [INFO][utils] Output directory: ../output/2023-02-21_12-06-52
2023-02-21 12:06:52,283 [INFO][utils] Python version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]
2023-02-21 12:06:52,283 [IN

In [10]:
from src.ice_cube.data_loader import make_test_dataloader
from src.ice_cube.model import load_pretrained_model
from src.ice_cube.submission import to_submission_df

graphnet: INFO     2023-02-21 12:06:53 - get_logger - Writing log to logs/graphnet_20230221-120653.log


2023-02-21 12:06:53,154 [INFO] [logging] Writing log to logs/graphnet_20230221-120653.log
/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


graphnet: WARNING  2023-02-21 12:06:55 - warn_once - `icecube` not available. Some functionality may be missing.


2023-02-21 12:06:55,526 [WARNING] [logging] `icecube` not available. Some functionality may be missing.


In [11]:
test_loader = make_test_dataloader(c)

graphnet: WARNING  2023-02-21 12:06:55 - SQLiteDataset.warning - Removing the following (missing) truth variables: zenith, azimuth


2023-02-21 12:06:55,735 [WARNING] [logging] Removing the following (missing) truth variables: zenith, azimuth


In [12]:
model = load_pretrained_model(c)

graphnet: WARNING  2023-02-21 12:06:55 - SQLiteDataset.warning - Removing the following (missing) truth variables: zenith, azimuth


2023-02-21 12:06:55,742 [WARNING] [logging] Removing the following (missing) truth variables: zenith, azimuth
2023-02-21 12:06:55,747 [INFO] [instantiator] Created a temporary directory at /tmp/tmpa2ogxlyg
2023-02-21 12:06:55,748 [INFO] [instantiator] Writing /tmp/tmpa2ogxlyg/_remote_module_non_sriptable.py


In [13]:
results = model.predict_as_dataframe(
    gpus=[0],
    dataloader=test_loader,
    prediction_columns=model.prediction_columns,
    additional_attributes=[c.settings.index_name],
)

2023-02-21 12:06:55,841 [INFO] [rank_zero] GPU available: True (cuda), used: True
2023-02-21 12:06:55,842 [INFO] [rank_zero] TPU available: False, using: 0 TPU cores
2023-02-21 12:06:55,842 [INFO] [rank_zero] IPU available: False, using: 0 IPUs
2023-02-21 12:06:55,842 [INFO] [rank_zero] HPU available: False, using: 0 HPUs
2023-02-21 12:06:55,845 [INFO] [rank_zero] GPU available: True (cuda), used: True
2023-02-21 12:06:55,845 [INFO] [rank_zero] TPU available: False, using: 0 TPU cores
2023-02-21 12:06:55,846 [INFO] [rank_zero] IPU available: False, using: 0 IPUs
2023-02-21 12:06:55,846 [INFO] [rank_zero] HPU available: False, using: 0 HPUs
2023-02-21 12:06:58,270 [INFO] [cuda] LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 0it [00:00, ?it/s]

In [14]:
results.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   direction_x      3 non-null      float64
 1   direction_y      3 non-null      float64
 2   direction_z      3 non-null      float64
 3   direction_kappa  3 non-null      float64
 4   event_id         3 non-null      float64
dtypes: float64(5)
memory usage: 248.0 bytes


In [15]:
results.head()

,direction_x,direction_y,direction_z,direction_kappa,event_id
0,0.987893,-0.148705,0.044195,2.946700,2092.0
1,-0.551074,-0.254085,-0.794832,6.980622,7344.0
2,-0.164367,-0.986232,0.018184,86.766502,9482.0


In [16]:
submission_df = to_submission_df(results)

In [17]:
submission_df.to_csv("submission.csv")

In [18]:
submission_df.head()

,azimuth,zenith
event_id,,
2092.0,6.133779,1.526587
7344.0,3.573616,2.489527
9482.0,4.547245,1.552611
